<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/SLC_LasVegasExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import graphviz
import pydot
from io import StringIO

from sklearn.preprocessing import StandardScaler, scale
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, multilabel_confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical

# Las Vegas Dataset

In [ ]:
!git clone https://github.com/danielbauer1979/ML_656.git

Cloning into 'ML_656'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 337 (delta 122), reused 123 (delta 76), pack-reused 117
Receiving objects: 100% (337/337), 27.13 MiB | 20.52 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [ ]:
lasvegas = pd.read_csv('ML_656/LasVegasTripAdvisorReviews.csv')
lasvegas.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3


In [ ]:
numerics = list(lasvegas.select_dtypes(include=['int64']).columns)
numerics.remove('Hotel stars')
numerics.remove('Score')
factors = list(lasvegas.select_dtypes(include=['object']).columns)
factors.append('Hotel stars')
factors.remove('User country')
factors.remove('Hotel name')

In [ ]:
lasvegas_numcols = lasvegas[numerics]
lasvegas_faccols = lasvegas[factors]
dummies = pd.get_dummies(lasvegas_faccols, drop_first=True)
lasvegas_numcols_sc_0 = scale(lasvegas_numcols)
lasvegas_numcols_sc = pd.DataFrame(data=lasvegas_numcols_sc_0, columns = lasvegas_numcols.columns, index = dummies.index)
lasvegas_sc = pd.concat([lasvegas_numcols_sc, dummies], axis = 1)
lasvegas_sc = pd.concat([lasvegas_sc, lasvegas['Score']], axis =1)
lasvegas_sc = lasvegas_sc.rename(columns={"Score":"Y"})
lasvegas_sc.head()

,Nr. reviews,Nr. hotel reviews,Helpful votes,Hotel stars,Period of stay_Jun-Aug,Period of stay_Mar-May,Period of stay_Sep-Nov,Traveler type_Couples,Traveler type_Families,Traveler type_Friends,Traveler type_Solo,Pool_YES,Gym_YES,Tennis court_YES,Spa_YES,Casino_YES,Free internet_YES,Y
0,-0.495595,-0.502370,-0.386857,3,0,0,0,0,0,1,0,0,1,0,0,1,1,5
1,0.945905,0.207912,0.892215,3,0,0,0,0,0,0,0,0,1,0,0,1,1,3
2,-0.161914,-0.293464,-0.139295,3,0,1,0,0,1,0,0,0,1,0,0,1,1,5
3,-0.455553,-0.377026,-0.366227,3,0,1,0,0,0,1,0,0,1,0,0,1,1,4
4,-0.575678,-0.460589,-0.613789,3,0,1,0,0,0,0,1,0,1,0,0,1,1,4


In [ ]:
np.random.seed(42)
train, test = train_test_split(lasvegas_sc, test_size = 0.3)
X_train = train.drop(columns = ['Y']).values
y_train = train['Y'].values
X_test = test.drop(columns = ['Y']).values
y_test = test['Y'].values

## Neural, once again

In [ ]:
inputs = keras.Input(shape=(17,))
x = layers.Dense(3, activation="relu", name="dense_1")(inputs)
x = layers.Dense(2, activation="relu", name="dense_2")(x)
outputs = layers.Dense(1, name="predictions")(x)
lasvegas_reg_nnet = keras.Model(inputs=inputs, outputs=outputs)
lasvegas_reg_nnet.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.004),
  # Loss function to minimize
  loss='mean_squared_error',
  # List of metrics to monitor
)

In [ ]:
history_reg = lasvegas_reg_nnet.fit(
  X_train,
  y_train,
  batch_size=50,
  epochs=850
)

Epoch 1/850
8/8 [==============================] - 1s 2ms/step - loss: 24.8179
Epoch 2/850
8/8 [==============================] - 0s 2ms/step - loss: 22.5022
Epoch 3/850
8/8 [==============================] - 0s 2ms/step - loss: 20.6537
Epoch 4/850
8/8 [==============================] - 0s 2ms/step - loss: 19.2098
Epoch 5/850
8/8 [==============================] - 0s 2ms/step - loss: 18.0959
Epoch 6/850
8/8 [==============================] - 0s 2ms/step - loss: 17.2651
Epoch 7/850
8/8 [==============================] - 0s 2ms/step - loss: 16.6796
Epoch 8/850
8/8 [==============================] - 0s 2ms/step - loss: 16.2960
Epoch 9/850
8/8 [==============================] - 0s 2ms/step - loss: 16.0011
Epoch 10/850
8/8 [==============================] - 0s 2ms/step - loss: 15.7410
Epoch 11/850
8/8 [==============================] - 0s 2ms/step - loss: 15.5004
Epoch 12/850
8/8 [==============================] - 0s 2ms/step - loss: 15.2691
Epoch 13/850
8/8 [==============================]

In [ ]:
lasvegas_nnet_reg_preds = lasvegas_reg_nnet.predict(X_test)
lasvegas_nnet_reg_preds = lasvegas_nnet_reg_preds.reshape((152,))
np.corrcoef(lasvegas_nnet_reg_preds,y_test)[0,1] * np.corrcoef(lasvegas_nnet_reg_preds,y_test)[0,1]

5/5 [==============================] - 0s 3ms/step


8.524393688156497e-05

In [ ]:
lasvegas_nnet_reg_preds

array([4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 ,
       4.2627172, 4.330723 , 4.330723 , 3.7940264, 4.330723 , 2.1970503,
       4.330723 , 4.330723 , 2.6138768, 4.330723 , 4.330723 , 4.330723 ,
       4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.256992 ,
       4.330723 , 4.330723 , 4.330723 , 3.466807 , 4.330723 , 4.330723 ,
       3.332961 , 4.0531783, 3.4259977, 4.330723 , 4.330723 , 4.330723 ,
       2.0097816, 4.330723 , 3.489731 , 4.330723 , 4.330723 , 4.330723 ,
       4.330723 , 4.330723 , 4.1771827, 4.330723 , 4.330723 , 4.330723 ,
       4.330723 , 3.472494 , 2.824432 , 4.330723 , 2.305881 , 4.330723 ,
       4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 , 2.8512683,
       4.330723 , 4.330723 , 4.330723 , 3.2360706, 4.330723 , 4.330723 ,
       4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 ,
       4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 , 4.330723 ,
       3.0875828, 4.330723 , 4.2043157, 4.330723 , 

In [ ]:
lasvegas_nnet_reg_preds_clss = np.round(lasvegas_nnet_reg_preds)
lasvegas_nnet_reg_preds_clss

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 2., 4., 4., 3., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4., 3., 4., 3., 4.,
       4., 4., 2., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 3.,
       4., 2., 4., 4., 4., 4., 4., 4., 3., 4., 4., 4., 3., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4., 4., 4., 4., 4.,
       4., 4., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4.,
       4., 4., 4., 4., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 1., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4.],
      dtype=float32)

In [ ]:
confusion_matrix(y_test, lasvegas_nnet_reg_preds_clss)

array([[ 0,  0,  0,  3,  0],
       [ 0,  1,  1,  9,  0],
       [ 0,  1,  2, 15,  0],
       [ 1,  1,  3, 46,  0],
       [ 0,  2,  6, 61,  0]])

## Multinomial Regression

In [ ]:
multin_model = LogisticRegression(fit_intercept=True, multi_class='multinomial',max_iter=10000).fit(X_train,y_train)

In [ ]:
y_mnmod_pred = multin_model.predict(X_test)
y_mnmod_pred

array([5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5,
       4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 5, 5, 5, 5, 5, 4,
       5, 4, 4, 4, 5, 4, 4, 5, 4, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5, 4, 5, 5,
       5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 4, 4, 5, 4, 5, 3, 5, 5, 4, 5, 5, 5, 3, 4, 5, 4, 4, 5, 4,
       4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 4, 4, 5, 5, 5, 5, 5,
       4, 5, 5, 5, 5, 4, 5, 5, 3, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 5])

In [ ]:
y_test

array([5, 5, 4, 3, 5, 4, 5, 5, 4, 5, 5, 3, 4, 4, 5, 4, 5, 4, 5, 5, 4, 5,
       5, 5, 4, 5, 3, 5, 4, 4, 5, 5, 4, 5, 3, 2, 5, 4, 2, 4, 3, 4, 5, 4,
       5, 5, 3, 2, 4, 5, 3, 5, 4, 4, 5, 3, 5, 5, 5, 5, 3, 2, 4, 3, 4, 5,
       5, 3, 2, 2, 4, 3, 5, 3, 4, 4, 4, 5, 4, 4, 1, 5, 4, 5, 4, 5, 4, 5,
       4, 5, 5, 5, 1, 4, 5, 5, 5, 5, 2, 4, 4, 5, 5, 4, 5, 5, 2, 5, 5, 4,
       3, 4, 4, 5, 5, 5, 4, 4, 5, 4, 2, 5, 5, 5, 3, 4, 5, 4, 5, 5, 5, 5,
       3, 5, 5, 5, 5, 4, 4, 4, 4, 1, 3, 5, 4, 4, 2, 3, 2, 5, 4, 4])

In [ ]:
np.corrcoef(y_mnmod_pred,y_test)[0,1] * np.corrcoef(y_mnmod_pred,y_test)[0,1]

0.07780061193125624

In [ ]:
confusion_matrix(y_test, y_mnmod_pred)

array([[ 0,  0,  0,  3,  0],
       [ 0,  0,  0,  5,  6],
       [ 0,  0,  2,  8,  8],
       [ 0,  0,  2,  9, 40],
       [ 0,  0,  2, 10, 57]])

In [ ]:
multilabel_confusion_matrix(y_test, y_mnmod_pred)

array([[[149,   0],
        [  3,   0]],

       [[141,   0],
        [ 11,   0]],

       [[130,   4],
        [ 16,   2]],

       [[ 75,  26],
        [ 42,   9]],

       [[ 29,  54],
        [ 12,  57]]])

In [ ]:
np.corrcoef(y_mnmod_pred,y_test)[0,1] * np.corrcoef(y_mnmod_pred,y_test)[0,1]

0.07780061193125624

In [ ]:
accuracy_score(y_test, y_mnmod_pred)

0.4473684210526316

## Neural Nets

In [ ]:
dummy_y_train = to_categorical(y_train)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352 entries, 5 to 102
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Nr. reviews             352 non-null    float64
 1   Nr. hotel reviews       352 non-null    float64
 2   Helpful votes           352 non-null    float64
 3   Hotel stars             352 non-null    int64  
 4   Period of stay_Jun-Aug  352 non-null    uint8  
 5   Period of stay_Mar-May  352 non-null    uint8  
 6   Period of stay_Sep-Nov  352 non-null    uint8  
 7   Traveler type_Couples   352 non-null    uint8  
 8   Traveler type_Families  352 non-null    uint8  
 9   Traveler type_Friends   352 non-null    uint8  
 10  Traveler type_Solo      352 non-null    uint8  
 11  Pool_YES                352 non-null    uint8  
 12  Gym_YES                 352 non-null    uint8  
 13  Tennis court_YES        352 non-null    uint8  
 14  Spa_YES                 352 non-null    ui

In [ ]:
inputs = keras.Input(shape=(17,))
x = layers.Dense(3, activation="relu", name="dense_1")(inputs)
x = layers.Dense(2, activation="relu", name="dense_2")(x)
outputs = layers.Dense(6, activation='softmax', name="predictions")(x)
lasvegas_nnet = keras.Model(inputs=inputs, outputs=outputs)
lasvegas_nnet.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.005),
  # Loss function to minimize
  loss='categorical_crossentropy',
  # List of metrics to monitor
  metrics=['accuracy'],
)

In [ ]:
history = lasvegas_nnet.fit(
  X_train,
  dummy_y_train,
  batch_size=50,
  epochs=1000
)

Epoch 1/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.7983 - accuracy: 0.0256
Epoch 2/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.7438 - accuracy: 0.1591
Epoch 3/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.7052 - accuracy: 0.4006
Epoch 4/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.6687 - accuracy: 0.4489
Epoch 5/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.6312 - accuracy: 0.4489
Epoch 6/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.5851 - accuracy: 0.4489
Epoch 7/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.5391 - accuracy: 0.4489
Epoch 8/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.4911 - accuracy: 0.4489
Epoch 9/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.4518 - accuracy: 0.4489
Epoch 10/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.4165 - accuracy: 0.4489
Epoch 11/1000
8/8 [

In [ ]:
lasvegas_nnet_val_preds = lasvegas_nnet.predict(X_test)
lasvegas_nnet_val_preds_class = np.argmax(lasvegas_nnet_val_preds, axis = 1)
lasvegas_nnet_val_preds_class

5/5 [==============================] - 0s 3ms/step


array([5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 3, 4, 5, 5, 5, 5, 5, 4,
       5, 4, 5, 4, 5, 4, 3, 5, 3, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 3, 5, 5,
       5, 5, 4, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 4,
       5, 5, 5, 4, 5, 4, 4, 5, 4, 5, 5, 4, 5, 4, 5, 3, 4, 4, 3, 4, 5, 4,
       5, 5, 4, 5, 5, 5, 5, 4, 4, 5, 4, 5, 5, 5, 4, 4, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 3, 4, 5, 5, 3, 4, 4, 5, 3, 5, 5, 5, 5, 5, 5, 5])

In [ ]:
y_test

array([5, 5, 4, 3, 5, 4, 5, 5, 4, 5, 5, 3, 4, 4, 5, 4, 5, 4, 5, 5, 4, 5,
       5, 5, 4, 5, 3, 5, 4, 4, 5, 5, 4, 5, 3, 2, 5, 4, 2, 4, 3, 4, 5, 4,
       5, 5, 3, 2, 4, 5, 3, 5, 4, 4, 5, 3, 5, 5, 5, 5, 3, 2, 4, 3, 4, 5,
       5, 3, 2, 2, 4, 3, 5, 3, 4, 4, 4, 5, 4, 4, 1, 5, 4, 5, 4, 5, 4, 5,
       4, 5, 5, 5, 1, 4, 5, 5, 5, 5, 2, 4, 4, 5, 5, 4, 5, 5, 2, 5, 5, 4,
       3, 4, 4, 5, 5, 5, 4, 4, 5, 4, 2, 5, 5, 5, 3, 4, 5, 4, 5, 5, 5, 5,
       3, 5, 5, 5, 5, 4, 4, 4, 4, 1, 3, 5, 4, 4, 2, 3, 2, 5, 4, 4])

In [ ]:
accuracy_score(y_test, lasvegas_nnet_val_preds_class)

0.4342105263157895

In [ ]:
confusion_matrix(y_test, lasvegas_nnet_val_preds_class)

array([[ 0,  0,  0,  2,  1],
       [ 0,  0,  1,  4,  6],
       [ 0,  0,  3,  5, 10],
       [ 0,  0,  4, 12, 35],
       [ 0,  0,  2, 16, 51]])

In [ ]:
np.corrcoef(lasvegas_nnet_val_preds_class,y_test)[0,1] * np.corrcoef(lasvegas_nnet_val_preds_class,y_test)[0,1]

0.03224008990412419

In [ ]:
accuracy_score(y_test, lasvegas_nnet_reg_preds_clss)

0.3223684210526316